In [1]:
library(tidyverse)
library(tidymodels)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

In [28]:
data <- read_csv2("Absenteeism_at_work.csv")
colnames(data) <- make.names(colnames(data))

data_1 <- data |>
    #mutate_at(c("ID", "Reason.for.absence", "Month.of.absence", "Day.of.the.week", "Seasons", 
    #            "Disciplinary.failure", "Education", "Social.drinker", "Social.smoker"), as.factor) |>
    mutate_at(c("ID", "Reason.for.absence", "Month.of.absence", "Day.of.the.week", "Seasons", 
                "Disciplinary.failure", "Education", "Social.drinker", "Social.smoker"), as.factor) |>
    select(-Reason.for.absence,-Month.of.absence,-Day.of.the.week,-Seasons,-Hit.target,-Weight,-Height,-Work.load.Average.day,-Disciplinary.failure) |>
    arrange(ID)
#data_1

data_total_hours <- data_1 |>
    group_by(ID) |>
    summarize(sum(Absenteeism.time.in.hours)) |>
    select(-ID)

data_2 <- data_1 |>
    mutate(across(everything(), as.numeric)) |>
    group_by(ID) |>
 #   summarize(mean=(mean(ID), na.rm  = TRUE))
    summarize(across(Transportation.expense:Absenteeism.time.in.hours, mean))
#data_2

#all_data <- cbind(data_2, data_total_hours) |>
#        select(-Absenteeism.time.in.hours)
 #   summarize(mean(ID:Absenteeism.time.in.hours, na.rm = TRUE))
 
data_by_days <- data |>
    group_by(ID) |>
    summarize(n = n()) |>
    select(-ID)

all_data <- cbind(data_2, data_total_hours, data_by_days) |>
        select(-Absenteeism.time.in.hours) |>
        rename(Days.of.absence = n, Hours.of.absence = "sum(Absenteeism.time.in.hours)") |>
        #mutate(ID,Social.drinker,Social.smoker,Education, as.factor)
        mutate(ID=as_factor(ID),Social.drinker=as_factor(Social.drinker),
               Social.smoker=as_factor(Social.smoker),Education=as_factor(Education))

#data_total_hours
#data_1
all_data

ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Rows: 740 Columns: 21
── Column specification ────────────────────────────────────────────────────────
Delimiter: ";"
dbl (20): ID, Reason for absence, Month of absence, Day of the week, Seasons...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


ID,Transportation.expense,Distance.from.Residence.to.Work,Service.time,Age,Education,Son,Social.drinker,Social.smoker,Pet,Body.mass.index,Hours.of.absence,Days.of.absence
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<int>
1,235,11.0,14.0,37.0,3,1.0,1,1,1,29.0,121,23
2,235,29.0,12.0,48.0,1,1.0,1,2,5,33.0,25,6
3,179,51.0,18.0,38.0,1,0.0,2,1,0,31.0,482,113
4,118,14.0,13.0,40.0,1,1.0,2,1,8,34.0,0,1
5,235,20.0,13.0,43.0,1,1.0,2,1,0,38.0,104,19
6,189,29.0,13.0,33.0,1,2.0,1,1,2,25.0,72,8
7,279,5.0,14.0,39.0,1,2.0,2,2,0,24.0,30,6
8,231,35.0,14.0,39.0,1,2.0,2,1,2,35.0,0,2
9,228,14.0,16.0,58.0,1,2.0,1,1,1,22.0,262,8
